In [59]:
with open('./male.txt') as  f:
    #读取txt的所有行到列表中
    male_name_list = f.readlines()
    #去掉列表每一项的换行符
    male_name_list = [name.strip() for name in male_name_list]
print(male_name_list[0:5])

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot']


对数据做处理：
1.从txt文件中读入所有名字，并以列表存储，列表的每个元素都是一个字符串
2.每个字符串都由字母letter组成，统计所有出现过的字母
3.在本任务中，RNN网络每一次输入应该是一个字母，因此要对字母编码
4.编码的方式非常简单，按照顺序给每个字母配一个int,还要再加上起始符start、终止符end
5.将所有字符串形式的名字全部编码成数组形式，并在每个名字加上起始和结束标识的编码，例如“Adam”->[0,1,10,2,3,54]

In [61]:
from collections import Counter
import numpy as np
import json
letter = Counter()
for name in male_name_list:
    for char in name:
        letter.update(char)
'''
letter2num = {k:i+1 for i , k in enumerate(letter)}
letter2num['pad'] = 0
letter2num['start']=len(letter2num)
letter2num['end']=len(letter2num)
num2letter = {letter2num[k]:k for k in letter2num.keys()}
print(letter2num)
'''
with open('./letter2num_encode.json','r') as f:
    letter2num=json.load(f)
encode_names=[]
names_len = []
for i,name in enumerate(male_name_list):
    encode_name = [letter2num['start']] + [letter2num[c] for c in name] + [letter2num['end']]
    encode_names.append(encode_name)
    names_len.append(len(encode_name))
print(encode_names[1:3])
print(names_len.index(min(names_len)))

[[54, 1, 2, 5, 6, 7, 55], [54, 1, 8, 8, 9, 10, 55]]
45


在网络中，每个名字要是一维张量，shape=(17,)，第一个元素一定是54，表示开始，中间会有55，表示结束，55后面全是0，用来占位

In [5]:
#import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch
class NamesDataset(Dataset):
    def __init__(self,Names:list,max_len=15):   #max_len要求每个名字最多10个字母，不足10个字母在后面补pad，也就是0
        self.Names=Names
        #self.transform=transform
        self.length=len(Names)
        self.max_len=max_len
    def __getitem__(self, index):
        name_len = len(self.Names[index])
        return torch.LongTensor(self.Names[index]+[0]*(self.max_len+2-name_len)),name_len    #加2是考虑有起始和终止符
    def __len__(self):
        return self.length
train_data = NamesDataset(encode_names)
train_loader = DataLoader(train_data,batch_size=20, shuffle=True,pin_memory=True)
#print:tensor([[54, 47, 18, 21,  6, 55,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        #      [54, 50,  8, 14, 10,  2, 12, 55,  0,  0,  0,  0,  0,  0,  0,  0,  0]]

In [10]:
for i , (names,lens) in enumerate(train_loader):
    print(names[:3].dtype,lens[:3])
    break

torch.int64 tensor([11, 10, 10])


RNN的输入虽然说是一个序列，但其实每次真正拿进去网络和隐层h一起搞的只有一个token，在这里就是一个字母
目前的一个名字的表示已经转换为了一个shape=(12,)的一维张量，但每个字母只是一个int而已，这样当然不行
需要把int转换成tensor
考虑所有的字母，包括起始符、终止符和pad，一共有56个字母，即len(letter2num)=56
可以使用nn.embedding(56,100),让56个字母，每个都用一个shape=(100,)的一维张量表示
原本的names=(batch,max_len+2)经过embed作为网络的input=(batch,max_len+2,100)

In [708]:
import torch.nn as nn
class GeneratorName(nn.Module):
    def __init__(self,letter2num,letter_dim,dropout=0.3,hidden_size=100) -> None:
        super(GeneratorName,self).__init__()
        self.letter_count=len(letter2num)
        self.hidden_size=hidden_size
        self.letter2num=letter2num
        self.letter_dim=letter_dim
        #hidden最开始是56维的one-hot向量，将其经过线性层
        self.init_hidden=nn.Sequential(
            nn.Softmax(dim=1),
            nn.Linear(self.letter_count,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
        )
        self.embed_layer=nn.Embedding(self.letter_count,letter_dim)
        self.rnn=nn.GRU(letter_dim,self.hidden_size,1)      #rnn的输入是100，因为每个字母转换成了100维向量,输出是100维向量
        self.dropout=nn.Dropout(dropout)
        self.fc=nn.Linear(self.hidden_size,self.letter_count)   #用线性层将100维变成56维

    def init_hidden_state(self, names, names_len):
        #names=(batch,max_len+2),names_len=(batch,)
        batch_size, names_dim = names.size(0), names.size(1)

        # 按照names的长短排序
        sorted_names_lens, sorted_nams_indices = torch.sort(names_len, 0, True)
        sorted_names = names[sorted_nams_indices]

        #hidden_state = torch.zeros(batch_size,self.letter_count).to(sorted_names.device)    #隐层状态的shape=(1,batch,100)
        hidden_state = torch.eye(self.letter_count).to(sorted_names.device)
        first_letters=sorted_names[:,1]
        """
        for i in range(batch_size):
            hidden_state[i , first_letters[i]] = 1
        """
        hidden_state = hidden_state[first_letters]
        hidden_state=self.init_hidden(hidden_state).unsqueeze(0)
        return  sorted_names, sorted_names_lens, hidden_state

    def forward_step(self, current_letter,hidden_state):
        #current_letter的维度是(batch,100)
        current_letter=current_letter.unsqueeze(0)      #让他的维度变成(1,batch,100)
        preds, hidden_state = self.rnn(current_letter, hidden_state)  #输出就是(1,batch,100),
        preds=self.fc(self.dropout(preds.squeeze(0)))       #变成(batch,56)
        return preds,hidden_state
    
    def forward(self,names,names_len):    #names.shape=(batch,max_len+2)
        sorted_names, sorted_names_lens, hidden_state=self.init_hidden_state(names,names_len)
        batch_size=sorted_names.size(0)
        lengths = sorted_names_lens.cpu().numpy() - 1
        input=self.embed_layer(sorted_names)    #sorted_names的shape=(batch,17)->input.shape=(batch,17,100)
        predictions = torch.zeros(batch_size, lengths[0], len(self.letter2num)).to(sorted_names.device)
        for step in range(lengths[0]):
            #（3）解码
            #（3.1）模拟pack_padded_sequence函数的原理，获取该时刻的非<pad>输入
            real_batch_size = np.where(lengths>step)[0].shape[0]
            preds, hidden_state = self.forward_step(
                            input[:real_batch_size, step, :],
                            hidden_state[:, :real_batch_size, :].contiguous())            
            # 记录结果
            predictions[:real_batch_size, step, :] = preds

        return predictions,sorted_names,lengths
    
    def add_noise2hidden(self,hidden_state,rand_ratio,n):
        noise=torch.randn(1,1,100)/(rand_ratio**n)
        return hidden_state+noise

    def predict(self,input_str,rand_ratio:int=5):
        output=[]
        #ell  -->ellen
        encode_str=[self.letter2num['start']]+[self.letter2num[letter] for letter in input_str]     #[start,..,..,..]
        length=torch.tensor([len(encode_str)])  #[4]
        input=torch.tensor(encode_str).unsqueeze(0)     #shape=(1,4)
        input, _, hidden_state=self.init_hidden_state(input,length)     #初始化隐状态
        count=0     #用来计数，产生了多少个字母
        embed_input=self.embed_layer(input)     #shape=(1,4,100)
        while(count<length[0]):
            if(count==length[0]-1):             #如果是最后一个已知输入产生的输出，即下一轮将使用此次输出作为输入的时候
                print('加入噪声')
                #hidden_state = self.add_noise2hidden(hidden_state,rand_ratio,count)
                #hidden_state=hidden_state+(torch.rand(1,1,100)-1)/(rand_ratio**count)  幂指数
                #hidden_state=hidden_state+(torch.rand(1,1,100))/(rand_ratio*count)     线性
            preds, hidden_state = self.forward_step(embed_input[:,count,:],hidden_state.contiguous())
            #preds的shape=(1,56)
            print(preds[0].topk(5)[0])
            output.append(preds[0].topk(1)[1].item())
            count = count+1
            if(count<length[0]):
                if(preds[0].topk(1)[1].item()==encode_str[count]):     #topk(1)[1]表示第一大的元素的索引
                    #print('第%d预测正确'%count)
                    pass
                else:
                    output[-1]=encode_str[count]
        while(output[-1]!=self.letter2num['end']):
            print('加入噪声')
            hidden_state=hidden_state+(torch.rand(1,1,100))/(rand_ratio*count)

            generated_letter=torch.tensor([output[-1]]).unsqueeze(0)  #shape=(1,1)
            embed_letter=self.embed_layer(generated_letter)     #shape=(1,1,100)
            preds, hidden_state = self.forward_step(embed_letter[0],hidden_state.contiguous())
            print(preds[0].topk(5)[0])
            output.append(preds[0].topk(1)[1].item())
            count=count+1
        #生成结束，丢掉结束符
        output=output[:-1]
        #将数字翻译成字母
        #print(output)
        output=[list(self.letter2num.keys())[list(self.letter2num.values()).index(n)] for n in output]
        name='可以起名为：'+''.join(output)
        return name

In [45]:
from torch.nn.utils.rnn import pack_padded_sequence
#构建损失函数
class PackedCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(PackedCrossEntropyLoss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, predictions, targets, lengths):
        """
        参数：
            predictions：按长度排序过预测结果   (batch,17,56)
            targets：按长度排序过的真实值       (batch,17)
            lengths：长度
        """
        #predictions经过pack以后会变成(real_batch,56),real_batch取决于lengths
        #targets经过pack以后会变成(real_batch,1),对应prediction56维匹配一个字母编号
        predictions = pack_padded_sequence(predictions, lengths, batch_first=True)[0]
        targets = pack_padded_sequence(targets, lengths, batch_first=True)[0]
        return self.loss_fn(predictions, targets) 

有了损失函数以后，可以实例化一个网络，直接训练
将数据丢进网络，将输出和真实值给损失函数，反传，选择优化器优化

In [709]:
model=GeneratorName(letter2num,letter_dim=100)
model.load_state_dict(torch.load('./male_name_generator_better.pt'))

<All keys matched successfully>

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
loss_fn=PackedCrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
model.train()
epoch=1
for e in range(epoch):
    for i,(names,names_len) in enumerate(train_loader):
        optimizer.zero_grad()
        names=names.to(device)
        names_len=names_len.to(device)
        predictions,sorted_names,lengths=model(names,names_len)
        loss=loss_fn(predictions,sorted_names[:,1:],lengths)
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        loss.backward()
        optimizer.step()
        if(i%100==99 and e%50==0):
            print('epoch %d, step %d: loss=%.2f' % (e+1,i+1, loss.cpu()))
test=torch.tensor([[54,  1, 14,  8,  9,  5, 11, 55,  0,  0,  0,  0,  0,  0,  0,  0,  0],
                    [54, 43, 17, 11,  2, 26,  4, 14, 18, 55,  0,  0,  0,  0,  0,  0,  0]]).to(device)
l=torch.tensor([8,10]).to(device)
output,_,l=model(test,l)
print(output[0,:].topk(1)[1].squeeze(1))
#print(output[1,:].topk(1)[1])

epoch 1, step 100: loss=0.97
tensor([43, 13, 11,  2, 26,  4,  6, 18, 55], device='cuda:0')


In [710]:
model.to('cpu')
model.eval()
#name=input('请输入前若干字母来起英文名：')
#model.predict(name)
model.predict('M',5)

tensor([19.1680, 10.5689, 10.0685,  9.7947,  9.0448], grad_fn=<TopkBackward0>)
加入噪声
tensor([9.0989, 7.8994, 7.8681, 7.7445, 6.6455], grad_fn=<TopkBackward0>)
加入噪声
tensor([6.1983, 4.9830, 4.3157, 4.2806, 3.8948], grad_fn=<TopkBackward0>)
加入噪声
tensor([5.0666, 4.7217, 4.6972, 4.4640, 4.1620], grad_fn=<TopkBackward0>)
加入噪声
tensor([6.8447, 6.6583, 6.1654, 5.5612, 5.0130], grad_fn=<TopkBackward0>)
加入噪声
tensor([6.2451, 5.7855, 5.5841, 4.2721, 3.8921], grad_fn=<TopkBackward0>)


'可以起名为：Marco'

In [14]:
torch.save(model.state_dict(),'./male_name_generator_better.pt')

In [26]:
from torch.nn.utils.rnn import pack_padded_sequence
pre=torch.tensor([[[0.1,0.4,0.5,0.6],[0.1,0.2,0.4,0.6],[0.7,0.8,0.1,0.1],[0.9,0,0,0]],[[0.5,0.4,0.3,0.1],[0.8,0.8,0.8,0.6],[0,0,0,0],[0,0,0,0]]])
targer=torch.tensor([[1,2,3,2],[3,2,0,0]])
len=[4,2]
pre=pack_padded_sequence(pre,len,batch_first=True)[0]
targer=pack_padded_sequence(targer,len,batch_first=True)[0]
print(pre,targer)
loss_fuc=nn.CrossEntropyLoss()
loss=loss_fuc(pre,targer)
print(loss)

tensor([[0.1000, 0.4000, 0.5000, 0.6000],
        [0.5000, 0.4000, 0.3000, 0.1000],
        [0.1000, 0.2000, 0.4000, 0.6000],
        [0.8000, 0.8000, 0.8000, 0.6000],
        [0.7000, 0.8000, 0.1000, 0.1000],
        [0.9000, 0.0000, 0.0000, 0.0000]]) tensor([1, 3, 2, 2, 3, 2])
tensor(1.5260)


11月13日的更新：
目前仍在训练阶段，训练的效果是一个名字输入网络后，可以得到输出，输出的后几位和原输入对得上 
说明最后一个字母能预测对
但是前面几个字母很难对，原因显然，因为第一个字母的产生只是依赖输入的起始符号，隐状态又是随机初始化不具有任何信息

因此我修改了隐状态生成的方法，我利用输入名字的第二字位置，也就是首字母对应的数字编码
让其成为隐状态的one-hot编码，是56维向量
然后经过softmax、线性、relu、线性转换成100维向量，这样理论上网络通过start起始符预测首字母时考虑了原输入真正的首字母

从这个网络的功能来看，因为它是要根据给出的一个或多个字母来预测名字，在生成第一个字母，如果不考虑输入的首字母，仅仅依赖起始符和
没有任何信息的隐状态，生成首字母是完全没有依据的，如果不是输入的首字母，很有可能会极大影响后续字母生成
就算生成对应的首字母，也可能不是大写的。第一个起始字母只要预测准确，就算是给后面的预测提供了方向

从训练的结果来看，在没有加这个方法时，在500个epoch以上loss也是在1.4到1.6中徘徊，使用这个方法后，300个epoch就出现了1.2的loss，
在约1000epoch的训练后，观察其中10个epoch的每个的100的step的loss，肉眼估计在1.2左右，更有两次1.07和1.08

随便将两个名字丢进网络中，产生的结果是：首字母能够准确预测，最后3个以上(包含结束符)字母也能准确预测，一般错误的是第二第三个，可以理解，
哪怕让人来起名字，只知道第一个字母是A，也不可能完全预测是Ann，Alb，Aus等，但后面几个能预测对，恰恰说明网络具有记忆能力，能够根据前文和当前输入预测下一个字母